**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [1]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 4.7 MB/s 
     |████████████████████████████████| 1.0 MB 46.2 MB/s 
     |████████████████████████████████| 591 kB 52.8 MB/s 


**Import các thư viện cần thiết**

In [12]:
import os
import re
import math
import numpy as np
from scipy.spatial import distance
from underthesea import text_normalize
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

**Lấy dữ liệu các tài liệu/văn bản mẫu từ**: https://github.com/HUTECH-OpenCourseWare/IRS.git

In [13]:
!git clone https://github.com/HUTECH-OpenCourseWare/IRS.git

fatal: destination path 'IRS' already exists and is not an empty directory.


**Tiến hành thử nghiệm với danh sách các tài liệu/văn bản thuộc các chủ đề khác nhau**

In [14]:
# Chọn danh sách các chủ đề của tài liệu/văn bản cho thử nghiệm
topics = [
    'the-thao',
    'du-lich',
    'khoa-hoc'
]

number_of_topics = len(topics)

# Với mỗi chủ đề - chúng ta sẽ giới hạn số lượng tài liệu/văn bản 
# Nhằm đảm bảo đồng đều về số lượng tài liệu/văn bản cho từng chủ đề
limit_doc_per_topic = 5

# Tạo một tập dữ liệu thử nghiệm gồm các tài liệu/văn bản thuộc về 2-3 chủ đề
# Cấu trúc dữ liệu dạng list - lưu thông tin danh sách các tài liệu/văn bản thuộc chủ đề khác nhau
# Mỗi tài liệu/văn bản sẽ tổ chức dạng 1 tuple với: (topic, nội_dung_văn_bản, danh_sách_token)
D = []

**Tiến hành viết một số hàm hỗ trợ cho việc đọc dữ liệu, xử lý và tách từ trong tiếng Việt.**

In [15]:
# Đọc danh sách các từ stopwords trong tiếng Việt
vn_stopwords_file_path = '/content/IRS/data/stopwords/vietnamese-stopwords.txt'
stopwords = []
with open(vn_stopwords_file_path, 'r', encoding='utf-8') as f:
  for line in f:
    line = line.strip()
    stopwords.append(line)

# Viết hàm tiền xử lý và tách từ tiếng Việt
def preprocess(doc):
  # Tiến hành xử lý các lỗi từ/câu, dấu câu, v.v. trong tiếng Việt với hàm text_normalize
  normalized_doc = text_normalize(doc)
  # Tiến hành tách từ
  tokens = word_tokenize(normalized_doc)
  # Lọc stopwords
  non_stopword_tokens = []
  for token in tokens:
    if token not in stopwords:
      non_stopword_tokens.append(token)
  # Tiến hành kết hợp các từ ghép trong tiếng Việt bằng '_'
  combined_tokens = [token.replace(' ', '_') for token in non_stopword_tokens]
  return (normalized_doc, combined_tokens)

# Viết hàm lấy danh sách các văn bản/tài liệu thuộc các chủ đề khác nhau
def fetch_doc_by_topic(topic, limit = 10):
  data_root_dir_path = '/content/IRS/data/vnexpress/{}'.format(topic)
  docs = []
  doc_count = 0
  for file_name in os.listdir(data_root_dir_path):
    if doc_count >= limit:
      break
    file_path = os.path.join(data_root_dir_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
      lines = []
      for line in f:
        line = line.lower().strip()
        lines.append(line)
    doc = " ".join(lines)
    clean_doc = re.sub('\W+',' ', doc)
    (normalized_doc, tokens) = preprocess(clean_doc)
    docs.append((topic, normalized_doc, tokens))
    doc_count+=1
  return docs

**Tiến hành tạo tập dữ liệu thử nghiệm với các tài liệu/văn bản thuộc danh sách chủ đề [topics] đã lựa chọn bên trên**

In [16]:
# Cấu trúc dữ liệu dictionary để lưu thông tin chủ đề-tài liệu, nhằm hỗ trợ cho việc tìm kiếm nhanh
topic_doc_idxes_dict = {}
doc_idx_topic_dict = {}

# Duyệt qua từng chủ đề
doc_idx = 0
for topic in topics:
  current_topic_docs = fetch_doc_by_topic(topic, limit_doc_per_topic)
  topic_doc_idxes_dict[topic] = []
  for (topic, normalized_doc, tokens) in current_topic_docs:
    topic_doc_idxes_dict[topic].append(doc_idx)
    doc_idx_topic_dict[doc_idx] = topic
    doc_idx+=1
  D += current_topic_docs

doc_size = len(D)

print('Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [{}]'.format(doc_size))
for topic in topic_doc_idxes_dict.keys():
  print(' - Chủ đề [{}] có [{}] tài liệu/văn bản.'.format(topic, len(topic_doc_idxes_dict[topic])))

Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [15]
 - Chủ đề [the-thao] có [5] tài liệu/văn bản.
 - Chủ đề [du-lich] có [5] tài liệu/văn bản.
 - Chủ đề [khoa-hoc] có [5] tài liệu/văn bản.


**Tiến hành biến đổi các tài liệu/văn bản trong tập (D) về dạng các TF-IDF vectors - trong bài thực hành này chúng ta sẽ sử dụng thư viện Scikit-Learn (TfidfVectorizer) https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html**

In [17]:
# Khởi tạo đối tượng TfidfVectorizer
vectorizer = TfidfVectorizer()

# Chúng ta sẽ tạo ra một tập danh sách các tài liệu/văn bản dạng list đơn giản để thư viện Scikit-Learn có thể đọc được
sk_docs = []

# Duyệt qua từng tài liệu/văn bản có trong (D)
for (topic, normalized_doc, tokens) in D:
  # Chúng ta sẽ nối các từ/tokens đã được tách để làm thành một văn bản hoàn chỉnh
  text = ' '.join(tokens)
  sk_docs.append(text)

# Tiến hành chuyển đổi các tài liệu/văn bản về dạng các TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(sk_docs)
print(f'Kích thước ma trận tf-idf: [{tfidf_matrix.shape}]')

# Chuyển ma trận tfidf_matrix từ dạng cấu trúc thưa sang dạng đầy đủ để thuận tiện cho việc tính toán
tfidf_matrix = tfidf_matrix.todense().tolist()

Kích thước ma trận tf-idf: [(15, 1843)]


**Tiến hành chuẩn bị tập dữ liệu cho bài toán gom cụm với thuật toán Hierarchical Agglomerative Clustering (HAC), với đầu vào sẽ là danh sách các tài liệu/văn bản đã được chuyển đổi về dạng các vector TF-IDF**

In [18]:
# Khởi tạo một danh sách chứa thông tin về mã định danh tài liệu (doc_idx) và tf-idf vector tương ứng của nó
docs = []
for doc_idx, doc_tfidf_vector in enumerate(tfidf_matrix):
  docs.append((doc_idx, doc_tfidf_vector))

**Tiến hành viết các hàm tính độ đo tương đồng và các hướng tiếp cận: liên kết đơn (single link), liên kết hoàn chỉnh (complete link) và trung bình nhóm (group average)**

**Trong đó việc xác định mức độ tương đồng giữa hai cụm ($c_{i}$) và ($c_{j}$) theo các hướng tiếp cận:**
*   **Liên kết đơn (single link):** 
## $$sim(c_{i},c_{j}=\max_{x∈c_{i},y∈c_{j}} sim(x,y)$$
*   **Liên kết hoàn chỉnh (complete link):**
## $$sim(c_{i},c_{j}=\min_{x∈c_{i},y∈c_{j}} sim(x,y)$$
*   **Trung bình nhóm (group average):**
### $$sim(c_{i},c_{j})=\frac{1}{|c_{i}∪c_{j}|(|c_{i}∪c_{j}|-1)}\sum_{x∈c_{i}∪c_{j}}\sum_{y∈c_{i}∪c_{j}:x \neq y}sim(x,y)$$



In [19]:
# Tính khoảng cách Euclid giữa vectors biểu diễn tài liệu/văn bản (a) và (b)
def euclid_distance(a, b):
    return distance.euclidean(a, b)

# Tính mức độ tương đồng giữa vectors biểu diễn tài liệu/văn bản (a) và (b)
def cosine_sim(a, b):
  return 1 - distance.cosine(a, b)

# Tính mức độ tương đồng lớn nhất của hai tài liệu/văn bản trong hai cụm (ci) và (cj) 
def single_link(ci, cj):
    # Vì áp dụng khoảng cách Euclid nên chúng ta sẽ lấy khoảng cách nhỏ nhất giữa hai tài liệu/văn bản có trong hai cụm (ci) và (cj) - min()
    # Ngược lại nếu dùng tương đồng cosine thì ta sẽ lấy giá trị lớn nhất - max()
    return min([euclid_distance(vi[1], vj[1]) for vi in ci for vj in cj])

# Tính mức độ tương đồng nhỏ nhất của hai tài liệu/văn bản trong hai cụm (ci) và (cj) 
def complete_link(ci, cj):
    # Tương tự như với tiến cận single link chúng ta sẽ lấy khoảng cách lớn nhất của hai tài liệu/văn bản có trong hai cụm (ci) và (cj) - max()
    # Ngược lại nếu dùng tương đồng cosine thì ta sẽ lấy giá trị lớn nhất - min()
    return max([euclid_distance(vi[1], vj[1]) for vi in ci for vj in cj])

# Tính trung bình khoảng cách giữa tất cả các cặp tài liệu/văn bản trong hai cụm (ci) và (cj)
def group_average(ci, cj):
    distances = [euclid_distance(vi[1], vj[1]) for vi in ci for vj in cj]
    return sum(distances) / len(distances)

**Cài đặt thuật toán Hierarchical Agglomerative Clustering (HAC)**

In [20]:
class HAC_Algorithm:
    def __init__(self, docs, k):
        self.docs = docs
        self.N = len(docs)
        self.k = k
        # Chúng ta có thể thử qua nhiều cách tiếp cận khác nhau: single link, complete link và group average
        self.measure = single_link
        self.clusters = self.init_clusters()

    # Hàm khởi tạo danh sách các cụm ban đầu
    def init_clusters(self):
         # Ban đầu thì mỗi tài liệu/văn bản sẽ được gán vào 1 cụm riêng biệt
        return {cluster_idx: [(doc_idx, doc_tfidf_vector)] for cluster_idx, (doc_idx, doc_tfidf_vector) in enumerate(self.docs)}

    # Hàm tìm cặp cụm (ci) và (cj) gần nhau nhất
    # Dựa trên các hướng tiếp cận: single link, complete link và group average - được định nghịa trong hàm measure
    def find_closest_clusters(self):
        # Ban đầu gán giá trị khoảng cách nhỏ nhất là một số rất lớn - dùng math.inf (vô cực)
        min_dist = math.inf
        closest_clusters = None
        # Lấy ra danh sách các index của các cụm từ [self.clusters]
        clusters_ids = list(self.clusters.keys())
        
        # Lần lượt lặp qua các cặp cụm (ci) và (cj) để lấy ra các cặp cụm gần nhau nhất
        for i, cluster_i in enumerate(clusters_ids[:-1]):
            for j, cluster_j in enumerate(clusters_ids[i+1:]):
                # Tính khoảng cách giữa hai cụm (ci) và (cj)
                dist = self.measure(self.clusters[cluster_i], self.clusters[cluster_j])
                # Kiểm tra xem khoảng cách này có phải là nhỏ nhất hay không ?
                if dist < min_dist:
                    # Nếu là khoảng cách nhỏ nhất thì cập nhật lại giá trị [min_dist]
                    # Cùng với thông tin cặp cụm (ci) và (cj) gần nhau nhất
                    min_dist, closest_clusters = dist, (cluster_i, cluster_j)
        return closest_clusters

    # Hàm trộn hai cụm (ci) và (cj) để tạo một cụm mới
    def merge_and_form_new_clusters(self, ci_id, cj_id):
        # Tạo một cụm mới tại ví trí đầu tiên [0] - trong [new_clusters]
        # Và dữ liệu của cụm là tập hợp danh sách các tài liệu/văn bản trong 2 cụm (ci) và (cj) gộp lại
        new_clusters = {0: self.clusters[ci_id] + self.clusters[cj_id]}
        # Lặp qua tất cả các cụm hiện tại có trong [self.clusters]
        for cluster_id in self.clusters.keys():
            # Nếu là hai cụm cũ (ci) và (cj) đang xét thì bỏ qua
            if (cluster_id == ci_id) | (cluster_id == cj_id):
                continue
            # Tiến hành bỏ thông tin các cụm cũ vào [new_clusters] với key tăng dần lên: 1, 2, 3, ...
            new_clusters[len(new_clusters.keys())] = self.clusters[cluster_id]
        return new_clusters

    # Hàm chính chạy thuật toán
    def run(self):
        # Kiểm tra xem số lượng cụm trong [self.clusters] đã đúng với số lượng cụm (k) kỳ vọng chưa
        # Nếu số lượng cụm trong [self.clusters] vẫn lớn hơn thì tiếp tục thực hiện gộp
        while len(self.clusters.keys()) > self.k:
            # Tiến hành tìm kiếm và trộn hai cụm (ci) và (cj) có khoảng cách gần nhau/độ tương đồng cao nhất 
            closest_clusters = self.find_closest_clusters()
            # Tiến hành trộn hai cụm (ci) và (cj) gần nhau nhất thành 1 cụm & cập nhật lại dữ liệu các cụm
            self.clusters = self.merge_and_form_new_clusters(*closest_clusters)

    # Xuất thông tin các cụm sau khi hoàn tất
    def show_cluster_info(self):
        for cluster_idx, cluster_data in self.clusters.items():
            print("Cụm (cluster) [{}]:".format(cluster_idx))
            for (doc_idx, doc_tfidf_vector) in cluster_data:
                print(f" - Chủ đề: [{doc_idx_topic_dict[doc_idx]}]/mã tài liệu: [{doc_idx}]")

**Tiến hành chạy thuật toán HAC với tập dữ liệu thuộc các chủ đề khác trong nhau tập [docs] - chúng ta sẽ lấy giá trị số lượng cụm kỳ vọng bằng với số lượng chủ đề - ví dụ: k = 3**

In [21]:
# Khai báo số lượng cụm kỳ vọng (k)
k = number_of_topics

# Khởi tạo & khai báo các tham số cho mô hình gom cụm HAC 
hac_algorithm = HAC_Algorithm(docs, k)

# Tiến hành chạy thuật toán HAC
print(f'Tiến hành chạy thuật toán HAC với tập dữ liệu: [{len(docs)}] tài liệu/văn bản - số cụm kỳ vọng (k) = [{k}]...')
hac_algorithm.run()

# Hoàn tất - xuất kết quả thông tin các cụm
print('Hoàn tất ! xuất thông tin các cụm...')
hac_algorithm.show_cluster_info()

Tiến hành chạy thuật toán HAC với tập dữ liệu: [15] tài liệu/văn bản - số cụm kỳ vọng (k) = [3]...
Hoàn tất ! xuất thông tin các cụm...
Cụm (cluster) [0]:
 - Chủ đề: [the-thao]/mã tài liệu: [0]
 - Chủ đề: [the-thao]/mã tài liệu: [4]
 - Chủ đề: [the-thao]/mã tài liệu: [2]
 - Chủ đề: [the-thao]/mã tài liệu: [1]
 - Chủ đề: [the-thao]/mã tài liệu: [3]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [10]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [12]
 - Chủ đề: [du-lich]/mã tài liệu: [8]
 - Chủ đề: [du-lich]/mã tài liệu: [6]
 - Chủ đề: [du-lich]/mã tài liệu: [5]
 - Chủ đề: [du-lich]/mã tài liệu: [7]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [13]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [11]
Cụm (cluster) [1]:
 - Chủ đề: [du-lich]/mã tài liệu: [9]
Cụm (cluster) [2]:
 - Chủ đề: [khoa-hoc]/mã tài liệu: [14]
